In [4]:
!pip install requests pandas sqlalchemy
!pip install --user psycopg2-binary
!pip install pymysql

In [27]:
# Install dependencies (run these in Jupyter/terminal before executing the script)
# !pip install requests pandas sqlalchemy pymysql psycopg2-binary

import os
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from datetime import date

# --- CONFIG ---
API_KEY = ""  # Replace with your OpenWeather API key
CITY = "London"

# MySQL connection settings
USER = ""
PASSWORD = ""  # Replace with your MySQL password
HOST = "127.0.0.1"
PORT = 3306
DB = ""

# Connection string (properly URL-encode password)
MYSQL_CONN_STRING = f"mysql+pymysql://{USER}:{quote_plus(PASSWORD)}@{HOST}:{PORT}/{DB}"


def fetch_weather(api_key: str, city: str) -> dict:
    """Fetch current weather from OpenWeather API."""
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key.strip(), "units": "metric"}

    r = requests.get(url, params=params, timeout=20)
    if r.status_code != 200:
        try:
            print("OpenWeather error payload:", r.json())
        except Exception:
            print("OpenWeather non-JSON response:", r.text)
        r.raise_for_status()

    data = r.json()
    return {
        "weather_date": date.today(),
        "city": city,
        "temp_c": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "description": data["weather"][0]["description"],
    }


def ensure_weather_table(engine) -> None:
    """Create table if not exists in MySQL."""
    ddl = """
    CREATE TABLE IF NOT EXISTS weather (
        weather_date DATE NOT NULL,
        city VARCHAR(100) NOT NULL,
        temp_c DECIMAL(5,2),
        humidity INT,
        description VARCHAR(255)
    )
    """
    with engine.begin() as conn:
        conn.execute(text(ddl))


def store_weather_to_db(weather_data: dict, conn_string: str) -> None:
    """Insert weather data into MySQL and display the table."""
    engine = create_engine(conn_string, pool_pre_ping=True)
    ensure_weather_table(engine)
    try:
        df = pd.DataFrame([weather_data])
        df.to_sql("weather", engine, if_exists="append", index=False)
        print(f"✅ Weather data for {weather_data['city']} stored successfully in MySQL.")
        
        # --- Display the created table ---
        with engine.connect() as conn:
            result = conn.execute(text("SELECT * FROM weather ORDER BY weather_date DESC LIMIT 10"))
            rows = result.fetchall()
            if rows:
                print("\n📊 Latest weather records in MySQL:")
                for row in rows:
                    print(row)
            else:
                print("⚠️ No data found in table.")
    finally:
        engine.dispose()


def main():
    weather = fetch_weather(API_KEY, CITY)
    store_weather_to_db(weather, MYSQL_CONN_STRING)


if __name__ == "__main__":
    main()


✅ Weather data for London stored successfully in MySQL.

📊 Latest weather records in MySQL:
(datetime.date(2025, 9, 23), 'London', Decimal('14.78'), 68, 'few clouds')
(datetime.date(2025, 9, 23), 'London', Decimal('14.74'), 68, 'few clouds')
